# imports

In [1]:
import requests as req
import time,random
import pandas as pd
from bs4 import BeautifulSoup as bp
import matplotlib.pyplot as  plot
import telebot
from mydata import token
import os

import warnings
warnings.filterwarnings("ignore")

# parsing

In [2]:
os.mkdir('fig_png')

# данный файл хранит список с юзер-агентами, которых в дальнейшем мы будем случайным образом выбирать для парсинга
with open ('us_ag.txt', encoding='utf-8') as file:
    lists = file.read().splitlines()

bot = telebot.TeleBot(token)

In [3]:
# определяем кол-во страниц, которые нужно парсить

def pagination_count():
    
    headers = {'user-agent' : random.choice(lists)}
    url = 'https://www.finanz.ru/indeksi/sostav/s&p_500'
    resp = req.get(url, headers=headers).text
    soup = bp(resp, 'lxml')
    count = int(soup.find('div', class_='finando_paging').find_all('a')[-2].text)
    
    return count

In [4]:
# вычленяем url
# имитируя человеческое поведение - делаем запросы с равновероятной периодичностью из диапазона 1-3 сек

def true_url():
    
    headers = {'user-agent' : random.choice(lists)}
    urls = []
    count = pagination_count()

    
    for page in range(count):
        
        url = f'https://www.finanz.ru/indeksi/sostav/s&p_500?p={page+1}'
        resp = req.get(url, headers=headers).text
        soup = bp(resp,'lxml')
        trs = soup.find_all('div', class_='content')[1].find('table').find_all('tr')[1:]
        
        for tr in trs:
            url = tr.find('td').find('a').get('href').strip().replace('aktsii', 'balans')
            url = 'https://www.finanz.ru/'+url
            urls.append(url)
        print(f'cтраница {page+1} обработана')
    
        time.sleep(random.uniform(1,3))
        
    return urls

In [5]:
# создаём csv файл с названиями компаний и их url - в дальнейшем это нам пригодиться

def one_year_change(count):
    
    finanz_data = pd.DataFrame()
    
    for page in range(count):

        headers = {'user-agent' : random.choice(lists)}   
        html = req.get(url=f'https://www.finanz.ru/indeksi/sostav/s&p_500?p={page+1}', headers=headers).text

        df = pd.read_html(html)
        df = df[2]

        count = len(df['Название ISIN'])
        
        for row in range(count):

            df['Название ISIN'][row] = df['Название ISIN'][row][:-12].strip().lower()
            df['1 год +/- %'][row] = df['1 год +/- %'][row].split(' ')[1].strip().replace(',','.')
           

        df['1 год +/- %'] = pd.to_numeric(df['1 год +/- %'])  
        df.set_index('Название ISIN', inplace=True)
        df.drop(df.columns[[0,1,2,3,4,5,6,7,9]], axis=1, inplace=True)
        df['1 год +/- %'] = pd.to_numeric(df['1 год +/- %'])
        
        finanz_data = pd.concat([finanz_data, df])
        
        
        print(f'cтраница {page+1} обработана')
        
        time.sleep(random.uniform(1,3))
        
    
    finanz_data['urls'] = true_url()
    finanz_data.to_csv('finanz_data', encoding='utf-8')


In [6]:
# создаём csv-файл с названиями компаний и их url

def create_finanz_data_file():
    one_year_change(pagination_count())
    
create_finanz_data_file()

# visualisation

In [7]:
# основная функция, которая визуализирует наши данные

def main_info(url):

    if url == 'https://www.finanz.ru//balans/leidos':
        print('Мы сожалеем, но по этому инструменту нет данных')

        
    try:
        headers = {'user-agent' : random.choice(lists)}   
        html=req.get(url, headers=headers).text

        debt = {}
        debtlist = req.get(url, headers=headers).text
        soup = bp(debtlist, 'lxml')
        tables = soup.find_all('table')[4:]

        for table in tables:
            try:
                try_table = table.find_all('tr')[-1].find('td').text.strip()
                if try_table == 'Коэффициент долговой нагрузки':
                    count = len(table.find('tr').find_all('th')[1:])
                    for year in range(count):
                        try:
                            debt[int(table.find('tr').find_all('th')[year+1].text)]=float(table.find_all('tr')[-1].find_all('td')[year+1].text.replace(',','.'))
                        except:
                            pass
                    debt = pd.DataFrame(debt.items(), columns=['YEARS', 'Debt'])

            except:
                pass




        eps = {}
        for table in tables:
            try:
                count = len(table.find('tr').find_all('th')[1:])
                try_table = table.find_all('tr')[1].find('td').text.strip()
                if try_table == 'Чистая прибыль на акцию':
                    for year in range(count):
                        try:
                            eps[int(table.find('tr').find_all('th')[year+1].text)]=float(table.find_all('tr')[1].find_all('td')[year+1].text.replace(',','.').replace('\xa0', ''))
                        except:
                            pass
                    eps = pd.DataFrame(eps.items(), columns=['YEARS', 'Чистая прибыль на акцию'])
            except:
                pass






        earnings = {}
        for table in tables:
            try:
                count = len(table.find('tr').find_all('th')[1:])
                try_table = table.find_all('tr')[1].find('td').text.strip()
                if try_table == 'Выручка':
                    for year in range(count):
                        try:
                            earnings[int(table.find('tr').find_all('th')[year+1].text)]=float(table.find_all('tr')[1].find_all('td')[year+1].text.replace(',','.').replace('\xa0', ''))
                        except:
                            pass
                    earnings = pd.DataFrame(earnings.items(), columns=['YEARS', 'Выручка'])
            except:
                pass
            
        
        ddds = {}
        for table in tables:
            try:
                count = len(table.find('tr').find_all('th')[1:])
                try_table = table.find_all('tr')[-2].find('td').text.strip()
                if try_table == 'Дивиденды на акцию':
                    for year in range(count):
                        try:
                            ddds[int(table.find('tr').find_all('th')[year+1].text)]=float(table.find_all('tr')[-2].find_all('td')[year+1].text.replace(',','.').replace('\xa0', ''))
                        except:
                            pass
                    ddds = pd.DataFrame(ddds.items(), columns=['YEARS', 'Дивиденды на акцию'])
            except:
                pass
       
      


        fig = plot.figure(f'{url.split("/")[-1]}.png', figsize=(16,7))
        fig.suptitle(url.split("/")[-1], fontsize=24)
        
        ax1 = fig.add_subplot(2,2,4)
        ax1.title.set_text('Чистая прибыль на акцию ($)')
        
        ax2 = fig.add_subplot(2,2,1)
        ax2.title.set_text('Выручка (млн, $)')
        
        ax3 = fig.add_subplot(2,2,3)
        ax3.title.set_text('Коэффициент долговой нагрузки (%)')
        
        ax4 = fig.add_subplot(2,2,2)
        ax4.title.set_text('Дивиденды на акцию ($)')
        
        ax1.plot('YEARS', 'Чистая прибыль на акцию', data=eps,  linewidth=2)
        ax1.set_xticks(eps['YEARS'])
        
        ax2.plot('YEARS', 'Выручка', data=earnings, color='cyan',  linewidth=2)
        ax2.set_xticks(earnings['YEARS'])
        
        ax3.plot('YEARS', 'Debt', data=debt, color='green',  linewidth=2)
        ax3.set_xticks(debt['YEARS'])
        
        ax4.plot('YEARS', 'Дивиденды на акцию', data=ddds, color='magenta',  linewidth=2)
        ax4.set_xticks(debt['YEARS'])
        
        
        plot.subplots_adjust(hspace = 0.3) # регулирует расстояние между графиками в фигуре, чтобы друг друга не задевали
        fig.savefig(f'fig_png/{url.split("/")[-1]}.png')
        fig_adress = f'fig_png/{url.split("/")[-1]}.png'
        
        return fig_adress
     
                
    except Exception as ex:
        print(ex)

In [8]:
# для каждой компании загрузим png-file, отражающие основные характеристики

def download_png():
    
    urls = true_url()
    list(map(main_info, urls))

download_png()

# interaction with the bot

In [9]:
# данная функция 'работает постоянно', благодаря bot.polling()

def get_data():
    
    @bot.message_handler(commands=['start'])
    def start_message(message):
        start = "Введите название компании на английском (не тикер),акции которой вы хотите просмотреть. \
        Если компаний, подходящих по название несколько - то данные выведутся о каждой из них." 
        bot.send_message(message.chat.id, start) 
        
    
    @bot.message_handler(content_types=['text'])
    def send_message(message):
        bot.send_message(message.chat.id, '...попробуем что-нибудь найти')
        name_stock = message.text.lower()
        df = pd.read_csv('finanz_data')
        count = 0

        print(name_stock)
        for row in range(len(df['Название ISIN'])):
            if name_stock in df['Название ISIN'][row]:
                count += 1
                
                name = df['urls'][row].split('/')[-1]
                fig_adress=f'fig_png/{name}.png'
                
                    
 
                url = df['urls'][row].replace('balans', 'profil-kompanii')
                bot.send_message(message.chat.id, url)
                bot.send_photo(message.chat.id, photo=open(fig_adress, 'rb'))

            
        if count == 0:
             bot.send_message(message.chat.id, 'данная акция не входит в s&p 500')

    bot.polling()

# autorun

In [ ]:
if __name__=='__main__':
    get_data()

рр
df
ит
шмгп
ыьлы
dfd
fdfd
fgf
арв
alpha
